In [1]:
import sys
print('Python: {}'.format(sys.version))
# scipy
import scipy
print('scipy: {}'.format(scipy.__version__))
# numpy
import numpy
print('numpy: {}'.format(numpy.__version__))
# matplotlib
import matplotlib
print('matplotlib: {}'.format(matplotlib.__version__))
# pandas
import pandas
print('pandas: {}'.format(pandas.__version__))
# scikit-learn
import sklearn
print('sklearn: {}'.format(sklearn.__version__))

Python: 2.7.13 |Anaconda 4.3.1 (64-bit)| (default, Dec 20 2016, 23:09:15) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
scipy: 0.18.1
numpy: 1.11.3
matplotlib: 2.0.0
pandas: 0.19.2
sklearn: 0.18.1


In [2]:
# Load libraries
import pandas as pd
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVC

In [3]:
import datetime
from datetime import date,time
import time

In [4]:
pd.options.mode.chained_assignment = None  # default='warn'

In [5]:
def time_num(date):
    intervalo = 30
    
    hora = date.hour
    minuto = date.minute
    
    num_minutes = hora*60 + minuto
    
    time_bin = num_minutes / intervalo
    hour_bin = num_minutes / 60
    min_bin = (time_bin * intervalo) % 60
    
    return ((hour_bin * 60 + min_bin + intervalo) / 2.0)/(60*24)

In [6]:
dataset = pd.read_csv('data/trip_train.csv')
weather = pd.read_csv('data/weather.csv')
station = pd.read_csv('data/station.csv')
#test = pd.read_csv('data/trip_test.csv')
#test.info()

In [7]:
station = station.loc[:,['id','lat','long']]

In [8]:
weather.date = pd.to_datetime(weather.date)

In [9]:
weather = weather.loc[:,['date','mean_temperature_f']].groupby('date').mean()
weather = pd.DataFrame(weather.to_records())

In [10]:
dataset = dataset.loc[(dataset.duration < 86400) & (dataset.duration > 60),:]

In [11]:
prueba = dataset.head(40000)
prueba.describe()

,id,duration,start_station_id,end_station_id,bike_id
count,40000.000000,40000.00000,40000.000000,40000.000000,40000.000000
mean,460869.256000,979.94955,58.034100,58.043125,428.361575
std,263885.890001,3147.20488,16.931549,16.990584,152.862854
min,4086.000000,61.00000,2.000000,2.000000,9.000000
25%,232081.250000,343.00000,50.000000,50.000000,334.000000
50%,457987.000000,517.00000,62.000000,62.000000,442.000000
75%,692537.000000,756.00000,70.000000,70.000000,546.000000
max,913416.000000,85049.00000,84.000000,84.000000,878.000000


In [12]:
prueba.start_date = pd.to_datetime(prueba.start_date)
#prueba.end_date = pd.to_datetime(prueba.end_date)

In [13]:
prueba['time_num'] = prueba.start_date.apply(lambda x: time_num(x))

In [14]:
prueba['date'] = prueba.start_date.apply(lambda x: datetime.datetime(x.year, x.month, x.day,0,0))

In [15]:
prueba = pd.merge(prueba,weather,on='date',how='inner')

In [16]:
prueba

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code,time_num,date,mean_temperature_f
0,907649,396,2015-08-27 08:36:00,Harry Bridges Plaza (Ferry Building),50,8/27/2015 8:43,Embarcadero at Sansome,60,187,Subscriber,94602,0.187500,2015-08-27,75.2
1,908115,486,2015-08-27 12:24:00,Embarcadero at Bryant,54,8/27/2015 12:32,Embarcadero at Vallejo,48,493,Subscriber,94105,0.260417,2015-08-27,75.2
2,907549,1027,2015-08-27 08:10:00,San Francisco Caltrain 2 (330 Townsend),69,8/27/2015 8:27,Market at Sansome,77,373,Subscriber,94105,0.177083,2015-08-27,75.2
3,907722,354,2015-08-27 08:53:00,Market at Sansome,77,8/27/2015 8:59,2nd at Folsom,62,395,Subscriber,94612,0.187500,2015-08-27,75.2
4,908290,209,2015-08-27 15:22:00,San Francisco Caltrain 2 (330 Townsend),69,8/27/2015 15:26,Townsend at 7th,65,313,Subscriber,94044,0.322917,2015-08-27,75.2
5,907615,849,2015-08-27 08:25:00,San Francisco Caltrain 2 (330 Townsend),69,8/27/2015 8:39,Clay at Battery,41,418,Subscriber,94303,0.177083,2015-08-27,75.2
6,907916,981,2015-08-27 09:49:00,Harry Bridges Plaza (Ferry Building),50,8/27/2015 10:05,Broadway St at Battery St,82,636,Subscriber,94114,0.208333,2015-08-27,75.2
7,908212,239,2015-08-27 13:58:00,Temporary Transbay Terminal (Howard at Beale),55,8/27/2015 14:02,Clay at Battery,41,529,Subscriber,94965,0.291667,2015-08-27,75.2
8,909018,698,2015-08-27 20:02:00,Embarcadero at Folsom,51,8/27/2015 20:14,San Francisco Caltrain (Townsend at 4th),70,576,Customer,85601,0.427083,2015-08-27,75.2
9,908705,414,2015-08-27 17:41:00,Market at 4th,76,8/27/2015 17:48,San Francisco Caltrain (Townsend at 4th),70,549,Subscriber,94303,0.375000,2015-08-27,75.2


In [17]:
prueba.describe()

,id,duration,start_station_id,end_station_id,bike_id,time_num,mean_temperature_f
count,40000.000000,40000.00000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000
mean,460869.256000,979.94955,58.034100,58.043125,428.361575,0.288172,62.207080
std,263885.890001,3147.20488,16.931549,16.990584,152.862854,0.094625,6.790143
min,4086.000000,61.00000,2.000000,2.000000,9.000000,0.010417,39.200000
25%,232081.250000,343.00000,50.000000,50.000000,334.000000,0.197917,57.600000
50%,457987.000000,517.00000,62.000000,62.000000,442.000000,0.291667,62.000000
75%,692537.000000,756.00000,70.000000,70.000000,546.000000,0.364583,67.800000
max,913416.000000,85049.00000,84.000000,84.000000,878.000000,0.500000,80.400000


In [18]:
prueba['hour_start'] = prueba.start_date.apply(lambda x: x.hour)
prueba['weekend'] = prueba.start_date.apply(lambda x: 1 if x.weekday() >= 5 else 0)

In [19]:
prueba

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code,time_num,date,mean_temperature_f,hour_start,weekend
0,907649,396,2015-08-27 08:36:00,Harry Bridges Plaza (Ferry Building),50,8/27/2015 8:43,Embarcadero at Sansome,60,187,Subscriber,94602,0.187500,2015-08-27,75.2,8,0
1,908115,486,2015-08-27 12:24:00,Embarcadero at Bryant,54,8/27/2015 12:32,Embarcadero at Vallejo,48,493,Subscriber,94105,0.260417,2015-08-27,75.2,12,0
2,907549,1027,2015-08-27 08:10:00,San Francisco Caltrain 2 (330 Townsend),69,8/27/2015 8:27,Market at Sansome,77,373,Subscriber,94105,0.177083,2015-08-27,75.2,8,0
3,907722,354,2015-08-27 08:53:00,Market at Sansome,77,8/27/2015 8:59,2nd at Folsom,62,395,Subscriber,94612,0.187500,2015-08-27,75.2,8,0
4,908290,209,2015-08-27 15:22:00,San Francisco Caltrain 2 (330 Townsend),69,8/27/2015 15:26,Townsend at 7th,65,313,Subscriber,94044,0.322917,2015-08-27,75.2,15,0
5,907615,849,2015-08-27 08:25:00,San Francisco Caltrain 2 (330 Townsend),69,8/27/2015 8:39,Clay at Battery,41,418,Subscriber,94303,0.177083,2015-08-27,75.2,8,0
6,907916,981,2015-08-27 09:49:00,Harry Bridges Plaza (Ferry Building),50,8/27/2015 10:05,Broadway St at Battery St,82,636,Subscriber,94114,0.208333,2015-08-27,75.2,9,0
7,908212,239,2015-08-27 13:58:00,Temporary Transbay Terminal (Howard at Beale),55,8/27/2015 14:02,Clay at Battery,41,529,Subscriber,94965,0.291667,2015-08-27,75.2,13,0
8,909018,698,2015-08-27 20:02:00,Embarcadero at Folsom,51,8/27/2015 20:14,San Francisco Caltrain (Townsend at 4th),70,576,Customer,85601,0.427083,2015-08-27,75.2,20,0
9,908705,414,2015-08-27 17:41:00,Market at 4th,76,8/27/2015 17:48,San Francisco Caltrain (Townsend at 4th),70,549,Subscriber,94303,0.375000,2015-08-27,75.2,17,0


In [22]:
prueba = prueba.loc[:,['duration','mean_temperature_f','time_num','weekend']]
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 549678 entries, 0 to 549960
Data columns (total 11 columns):
id                    549678 non-null int64
duration              549678 non-null int64
start_date            549678 non-null object
start_station_name    549678 non-null object
start_station_id      549678 non-null int64
end_date              549678 non-null object
end_station_name      549678 non-null object
end_station_id        549678 non-null int64
bike_id               549678 non-null int64
subscription_type     549678 non-null object
zip_code              544300 non-null object
dtypes: int64(5), object(6)
memory usage: 50.3+ MB


In [23]:
prueba

,duration,mean_temperature_f,time_num,weekend
0,396,75.2,0.187500,0
1,486,75.2,0.260417,0
2,1027,75.2,0.177083,0
3,354,75.2,0.187500,0
4,209,75.2,0.322917,0
5,849,75.2,0.177083,0
6,981,75.2,0.208333,0
7,239,75.2,0.291667,0
8,698,75.2,0.427083,0
9,414,75.2,0.375000,0


In [24]:
#start_station_name = pd.get_dummies(prueba['start_station_name'])
#end_station_name = pd.get_dummies(prueba['end_station_name'])

In [25]:
#print start_station_name.shape
#print end_station_name.shape

In [26]:
#prueba = prueba.drop('start_station_name',axis=1)
#prueba = prueba.drop('end_station_name',axis=1)
#prueba.shape

In [27]:
#prueba = prueba.join(start_station_name)

In [28]:
#columnas = [x +  '_end' for x in list(end_station_name.columns.tolist())]
#end_station_name.columns = columnas

In [29]:
#prueba = prueba.join(end_station_name)

In [30]:
#days = pd.get_dummies(prueba['weekday'])
#prueba = prueba.join(days)
#prueba = prueba.drop('weekday',axis=1)
#prueba

In [31]:
columnas = prueba.columns.tolist()
columnas = columnas[1:len(columnas)]
columnas.append('duration')
prueba = prueba[columnas]
len(prueba.columns.tolist())

4

In [32]:
from sklearn import preprocessing

In [33]:
array = prueba.values
X = array[:,0:len(prueba.columns.tolist())-1]
Y = array[:,len(prueba.columns.tolist())-1]
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)

In [34]:
seed = 7
scoring = 'accuracy'

In [35]:
#models = []
#models.append(('LR', LogisticRegression()))
#models.append(('LDA', LinearDiscriminantAnalysis()))
#models.append(('KNN', KNeighborsClassifier()))
#models.append(('CART', DecisionTreeClassifier()))
#models.append(('NB', GaussianNB()))
#models.append(('SVM', SVC()))
# evaluate each model in turn
#results = []
#names = []
#for name, model in models:
#    kfold = model_selection.KFold(n_splits=10, random_state=seed)
#    cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
#    results.append(cv_results)
#   names.append(name)
#    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
#    print(msg)

In [36]:
start_time = time.time()
models = []
#models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
#models.append(('SVM', SVC()))
# evaluate each model in turn
for name, model in models:
    model.fit(X_train,Y_train)
    predictions = model.predict(X_validation)
    msg = "%s -> Mean Squared Error: %f" % (name, mean_squared_error(Y_validation, predictions))
    print(msg)
print("--- %s seconds ---" % (time.time() - start_time))

LDA -> Mean Squared Error: 11776714.596875
KNN -> Mean Squared Error: 11871854.835875
CART -> Mean Squared Error: 14459878.236875
NB -> Mean Squared Error: 50604191.093625
--- 12.5915381908 seconds ---


In [40]:
start_time = time.time()
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, Y_train)
predictions = lda.predict(X_validation)
print Y_validation[0:15]
print predictions[0:15]
#print accuracy_score(Y_validation,predictions)
print(mean_squared_error(Y_validation, predictions))
print("--- %s seconds ---" % (time.time() - start_time))

[   487.   1076.    826.    526.    629.    526.  13383.    351.    256.
   2333.    463.    641.    366.    636.    383.]
[  485.   438.   405.   216.   387.   438.   469.   387.   405.   438.
   405.   387.  1054.   387.   438.]
11776714.5969
--- 2.18736600876 seconds ---
